In [6]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HUGGING_FACE_TOKEN")
login(token=hf_token)

In [7]:
from perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv() 

client = Perplexity() # PERPLEXITY_API_KEY is read from environment variable

search = client.search.create(
    query=[
      "What is Comet Browser?",
      "Perplexity AI",
      "Perplexity Changelog"
    ]
)

for result in search.results:
    print(f"{result.title}: {result.url}")

Introducing Comet: Browse at the speed of thought: https://www.perplexity.ai/hub/blog/introducing-comet
Browse at the speed of thought: https://www.perplexity.ai/comet/
Comet Browser Review 2025: Features, Pricing, Pros & Cons: https://efficient.app/apps/comet
Perplexity's AI-Native Browser Comet is HERE: https://www.youtube.com/watch?v=ri_bFrDp44M
Introducing Comet: Browse at the Speed of Thought: https://www.youtube.com/watch?v=YeldJ4UezDQ
Perplexity launches Comet, an AI-powered web browser: https://techcrunch.com/2025/07/09/perplexity-launches-comet-an-ai-powered-web-browser/
What Is Comet and Why Everyone's Talking About It: https://apidog.com/blog/what-is-comet/
Perplexity launches Comet browser to rival Google: https://timesofindia.indiatimes.com/technology/tech-news/perplexity-launches-comet-browser-to-rival-google-what-are-the-features-availability-and-benefits-of-comet/articleshow/124093855.cms
Comet: The AI browser gets personal: https://www.ibm.com/think/news/comet-perplexi

In [8]:
import json
from pprint import pprint

goal = "Analyze APPL stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create a detailed plan to achieve this goal:
    Goal: {goal}
    
    Available tools: web search, financial data API, news aggregator, sentiment analysis tool

    Create a plan with:
    1. Logical research steps (3-7 steps)
    2. Estimated time in minutes
    3. Required tools for each step
    4. Success criteria to evaluate completion
    
    Answer in JSON format:
    {{
        "goal": "...",
        "steps": ["step1", "step2", ...],
        "estimated_time": 30,
        "required_tools": ["tool1", "tool2", ...],
        "success_criteria": ["criteria1", "criteria2", ...]
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)


### Presenting the data generated

In [12]:

#print("Raw Response:\n", response.choices[0].message.content)
plan_data = json.loads(response.choices[0].message.content)

pprint(plan_data)

{'confidence': 0.75,
 'sentiment': 'Neutral',
 'stock': 'PLTR',
 'summary': "Palantir's stock shows strong revenue growth, with Q2 2025 "
            'revenues surpassing $1 billion and a 48% year-over-year increase, '
            "including a 93% rise in U.S. commercial revenue. The company's "
            'leadership remains optimistic about future expansion. However, '
            'its current valuation is very high, with a price-to-sales ratio '
            'around 132 and a P/E of nearly 378, suggesting the stock might be '
            'overvalued. While Palantir benefits from demand for AI and data '
            'analytics, this frothy valuation may limit upside potential in '
            'the near term, and some top investors express caution. Trading '
            'volume is below average recently, with price down slightly on the '
            'day but near its 52-week high, reflecting some consolidation. '
            'Overall, the fundamentals are positive, but valuation and r

### Sentiment analysis using perplexity API

#### Get information from FinTools

In [10]:
from src.financial_tools import StockTools, NewsTools

ticker = "PLTR"  # Example stock ticker
stock_info = StockTools.get_stock_info(ticker)
pprint(stock_info)

news_articles = NewsTools.get_latest_news(ticker, limit=5)
for article in news_articles:
    pprint(article)


{'52_week_high': 190.0,
 '52_week_low': 36.05,
 'avg_volume_30d': 72155560.0,
 'beta': 2.592,
 'company_summary': 'Palantir Technologies Inc. builds and deploys software '
                    'platforms for the intelligence community to assist in '
                    'counterterrorism investigations and operations in the '
                    'United States, the United Kingdom, and internationally. '
                    'It provides Palantir Gotham, a software platform, which '
                    'enables users to identify patterns hidden deep within '
                    'datasets, ranging from signals intelligence sources to '
                    'reports from confidential informants, as well as '
                    'facilitates the hand-off between analysts and operational '
                    'users, he',
 'current_price': 177.57000732421875,
 'dividend_yield': None,
 'historical_data': [{'Close': 177.1699981689453,
                      'Dividends': 0.0,
                      

#### Generate sentiment analysis and confidence score

In [11]:

goal = f"Analyze {ticker} stock for a potential investment."

prompt = f"""
    As an autonomous investment research agent, create short sentiment analysis
      about the stock based on the context and recent news.
      Describe the sentiment as Positive, Negative, or Neutral.
      Give a summary of your analysis, and a confidence score between 0 and 1.
    Goal: {goal}

    Context: {stock_info}

    Recent News: {news_articles}

    Answer in JSON format:
    {{
        "stock": "...",
        "sentiment": "Positive",
        "summary": "...",
        "confidence": 0.85
    }}
"""


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="sonar",
)

print("Raw Response:\n", response.choices[0].message.content)

Raw Response:
 {
  "stock": "PLTR",
  "sentiment": "Neutral",
  "summary": "Palantir's stock shows strong revenue growth, with Q2 2025 revenues surpassing $1 billion and a 48% year-over-year increase, including a 93% rise in U.S. commercial revenue. The company's leadership remains optimistic about future expansion. However, its current valuation is very high, with a price-to-sales ratio around 132 and a P/E of nearly 378, suggesting the stock might be overvalued. While Palantir benefits from demand for AI and data analytics, this frothy valuation may limit upside potential in the near term, and some top investors express caution. Trading volume is below average recently, with price down slightly on the day but near its 52-week high, reflecting some consolidation. Overall, the fundamentals are positive, but valuation and recent price performance warrant a cautious stance.",
  "confidence": 0.75
}
